# MOPI-HFRS Training on Google Colab

**Multi-Objective Personalized Interpretable Health-aware Food Recommendation System**

This notebook trains the MOPI-HFRS model on Google Colab with A100 GPU.

## Features:
- **CSV or PT file** data loading
- **Macro (7 nutrients)** or **All (16 nutrients)** benchmark support
- **Pareto multi-objective optimization** (BPR + Health + Diversity)
- **Health-aware graph structure learning**


## 1. Setup Environment


In [ ]:
# Check GPU
!nvidia-smi


In [ ]:
# Install dependencies
# PyTorch 2.8.0 is needed for torch-sparse/scatter compatibility
%pip install torch==2.8.0 torchvision torchaudio -q
%pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.8.0+cu121.html -q
%pip install torch-geometric -q
%pip install tqdm pandas scikit-learn -q


In [ ]:
# Mount Google Drive (if data is stored there)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Copy project files from Drive to Colab
# Adjust the path according to your Drive structure
!cp -r /content/drive/MyDrive/HFRSStudio /content/

# Set working directory
import os
os.chdir('/content/HFRSStudio')


## 2. Import Libraries and Check Environment


In [ ]:
import sys
sys.path.insert(0, '/content/HFRSStudio')

import torch
import numpy as np
from pathlib import Path

# Check PyTorch and CUDA
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


In [ ]:
from config import get_colab_config
from data.data_loader import HFRSDataset, HFRSDatasetFromPT
from models.mopi_hfrs import create_model
from train import train, set_seed


## 3. Configure Training


In [ ]:
# ============================================================
# DATA CONFIGURATION
# ============================================================

# Data source: CSV files or pre-processed PT file
USE_CSV = True  # Set to True to load from CSV files, False for PT file

# Benchmark type: 'macro' (7 nutrients) or 'all' (16 nutrients)
# - macro: calories, carbohydrates, protein, saturated fat, cholesterol, sugar, dietary fiber
# - all: macro + sodium, potassium, phosphorus, iron, calcium, folic acid, vitamin C, D, B12
BENCHMARK_TYPE = 'macro'

# Data paths
DATA_DIR = '/content/drive/MyDrive/MOPI-HFRS_gdrive/processed_data'
PT_FILE = f'{DATA_DIR}/benchmark_{BENCHMARK_TYPE}.pt'  # Only used if USE_CSV=False

# ============================================================
# TRAINING CONFIGURATION  
# ============================================================

config = get_colab_config()

# Data settings
config.data.data_dir = DATA_DIR
config.data.benchmark_type = BENCHMARK_TYPE
config.data.use_pt_file = not USE_CSV
config.data.pt_file = PT_FILE
config.data.train_ratio = 0.6
config.data.val_ratio = 0.2

# Training settings
config.training.epochs = 500
config.training.batch_size = 4096  # Large batch for A100
config.training.learning_rate = 1e-3
config.training.eval_every = 25
config.training.save_dir = '/content/drive/MyDrive/checkpoints/mopi_hfrs'
config.training.seed = 42

# Model settings
config.model.embedding_dim = 128
config.model.num_layers = 3
config.model.num_heads = 4
config.model.feature_threshold = 0.3

# Print configuration
print("="*60)
print("CONFIGURATION")
print("="*60)
print(f"Data source: {'CSV files' if USE_CSV else 'PT file'}")
print(f"Benchmark type: {BENCHMARK_TYPE}")
print(f"Data dir: {DATA_DIR}")
print(f"Train/Val/Test: {config.data.train_ratio}/{config.data.val_ratio}/{1-config.data.train_ratio-config.data.val_ratio}")
print()
print(f"Epochs: {config.training.epochs}")
print(f"Batch size: {config.training.batch_size}")
print(f"Learning rate: {config.training.learning_rate}")
print()
print(f"Embedding dim: {config.model.embedding_dim}")
print(f"Num layers: {config.model.num_layers}")
print(f"Num heads: {config.model.num_heads}")
print("="*60)


## 4. Train Model


In [ ]:
import time
from tqdm.auto import tqdm

print("Loading dataset...")
start_time = time.time()

if USE_CSV:
    # Load from CSV files
    dataset = HFRSDataset(
        data_dir=DATA_DIR,
        train_ratio=config.data.train_ratio,
        val_ratio=config.data.val_ratio,
        seed=config.training.seed,
        normalize=config.data.normalize_features,
        benchmark_type=BENCHMARK_TYPE
    )
else:
    # Load from PT file
    dataset = HFRSDatasetFromPT(
        pt_file=PT_FILE,
        train_ratio=config.data.train_ratio,
        val_ratio=config.data.val_ratio,
        seed=config.training.seed
    )

load_time = time.time() - start_time

print(f"\nDataset loaded in {load_time:.1f} seconds")
print(f"  Users: {dataset.num_users:,}")
print(f"  Foods: {dataset.num_foods:,}")
print(f"  User features: {dataset.user_features.shape}")
print(f"  Food features: {dataset.food_features.shape}")
print(f"  Train edges: {dataset.splits['train_edge_index'].shape[1]:,}")


## 5. Create Model and Train


In [ ]:
from models.mopi_hfrs import MOPI_HFRS
from utils.losses import bpr_loss, health_loss, diversity_loss
import torch.optim as optim
import random
import os

# Create model
model = MOPI_HFRS(
    num_users=dataset.num_users,
    num_foods=dataset.num_foods,
    user_feature_dim=dataset.user_features.shape[1],
    food_feature_dim=dataset.food_features.shape[1],
    embedding_dim=config.model.embedding_dim,
    num_layers=config.model.num_layers,
    num_heads=config.model.num_heads,
    feature_threshold=config.model.feature_threshold
)

# Move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
dataset = dataset.to(device)

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Model created on {device}")
print(f"  Total parameters: {num_params:,}")


In [ ]:
# Set seed for reproducibility
torch.manual_seed(config.training.seed)
np.random.seed(config.training.seed)
random.seed(config.training.seed)

# Create save directory
os.makedirs(config.training.save_dir, exist_ok=True)

# Optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=config.training.learning_rate, weight_decay=config.training.weight_decay)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=config.training.lr_decay_gamma)

# Get data tensors
feature_dict = dataset.get_feature_dict()
train_edge_index = dataset.splits['train_edge_index']
val_edge_index = dataset.splits['val_edge_index']
test_edge_index = dataset.splits['test_edge_index']

# Get positive/negative edges for signed graph learning
pos_edge_index = dataset.graph['user', 'eats', 'food'].pos_edge_index.to(device)
neg_edge_index = dataset.graph['user', 'eats', 'food'].neg_edge_index.to(device)

# Get tags for health loss
user_tags = dataset.user_tags
food_tags = dataset.food_tags
food_features = dataset.food_features

print(f"Train edges: {train_edge_index.shape[1]:,}")
print(f"Pos edges: {pos_edge_index.shape[1]:,}")
print(f"Neg edges: {neg_edge_index.shape[1]:,}")


In [ ]:
def sample_mini_batch(batch_size, edge_index, num_items):
    """Sample a mini-batch for training."""
    num_edges = edge_index.shape[1]
    indices = torch.randperm(num_edges, device=edge_index.device)[:batch_size]
    
    user_indices = edge_index[0, indices]
    pos_item_indices = edge_index[1, indices]
    neg_item_indices = torch.randint(0, num_items, (batch_size,), device=edge_index.device)
    
    return user_indices, pos_item_indices, neg_item_indices


# Training loop
best_val_loss = float('inf')
train_losses = []

print("\nStarting training...")
print("="*60)

for epoch in tqdm(range(1, config.training.epochs + 1), desc="Training"):
    model.train()
    
    # Forward pass
    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model(
        feature_dict, train_edge_index, pos_edge_index, neg_edge_index
    )
    
    # Sample mini-batch
    user_indices, pos_item_indices, neg_item_indices = sample_mini_batch(
        config.training.batch_size, train_edge_index, dataset.num_foods
    )
    
    # Get batch embeddings
    users_batch = users_emb_final[user_indices]
    users_0_batch = users_emb_0[user_indices]
    pos_items_batch = items_emb_final[pos_item_indices]
    pos_items_0_batch = items_emb_0[pos_item_indices]
    neg_items_batch = items_emb_final[neg_item_indices]
    neg_items_0_batch = items_emb_0[neg_item_indices]
    
    # Get batch tags
    user_tags_batch = user_tags[user_indices]
    pos_item_tags_batch = food_tags[pos_item_indices]
    neg_item_tags_batch = food_tags[neg_item_indices]
    
    # Get batch features for diversity
    pos_item_features_batch = food_features[pos_item_indices]
    
    # Compute losses
    loss_bpr = bpr_loss(
        users_batch, users_0_batch,
        pos_items_batch, pos_items_0_batch,
        neg_items_batch, neg_items_0_batch,
        lambda_val=config.training.lambda_val
    )
    
    loss_health = health_loss(
        user_tags_batch, pos_item_tags_batch, neg_item_tags_batch,
        users_batch, pos_items_batch, neg_items_batch
    )
    
    loss_div = diversity_loss(
        users_batch, pos_items_batch, pos_item_features_batch
    )
    
    # Combined loss (Pareto optimization can be added later)
    total_loss = loss_bpr + 0.1 * loss_health + 0.1 * loss_div
    
    # Backward
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    train_losses.append(total_loss.item())
    
    # Evaluation
    if epoch % config.training.eval_every == 0:
        model.eval()
        with torch.no_grad():
            val_users_emb, _, val_items_emb, _ = model(
                feature_dict, val_edge_index, pos_edge_index, neg_edge_index
            )
            val_user_idx = val_edge_index[0]
            val_item_idx = val_edge_index[1]
            val_scores = (val_users_emb[val_user_idx] * val_items_emb[val_item_idx]).sum(dim=1)
            val_loss = -torch.mean(torch.log(torch.sigmoid(val_scores) + 1e-8)).item()
        
        print(f"\nEpoch {epoch}: Train={total_loss.item():.4f}, "
              f"BPR={loss_bpr.item():.4f}, Health={loss_health.item():.4f}, "
              f"Div={loss_div.item():.4f}, Val={val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
            }, f"{config.training.save_dir}/best_model.pt")
            print(f"  -> Saved best model (val_loss={best_val_loss:.4f})")
    
    # Learning rate decay
    if epoch % config.training.lr_decay_step == 0:
        scheduler.step()

print("\n" + "="*60)
print("Training completed!")
print(f"Best validation loss: {best_val_loss:.4f}")


## 6. Generate Recommendations


In [ ]:
# Load best model
checkpoint = torch.load(f"{config.training.save_dir}/best_model.pt")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Loaded best model from epoch {checkpoint['epoch']}")

# Generate embeddings
with torch.no_grad():
    users_emb, items_emb = model.get_embeddings(
        feature_dict, train_edge_index, pos_edge_index, neg_edge_index
    )

print(f"User embeddings: {users_emb.shape}")
print(f"Item embeddings: {items_emb.shape}")

# Generate top-10 recommendations for sample users
print("\nSample Recommendations:")
print("="*60)

for user_idx in [0, 100, 1000, 5000, 10000]:
    if user_idx >= dataset.num_users:
        continue
        
    top_k_items, top_k_scores = model.recommend(
        user_idx, users_emb, items_emb, k=10
    )
    
    print(f"\nUser {user_idx}:")
    for i, (item_idx, score) in enumerate(zip(top_k_items, top_k_scores)):
        print(f"  {i+1}. Food {item_idx.item()}: score={score.item():.4f}")
